In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

image_test = '<PATH>'

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 21, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(21, 16, 5)
        #self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck', 'melanoma')

image = Image.open(image_test).convert('RGB')
PATH = './cifar_net.pth'

transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Adjust to your network's input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Adjust normalization parameters as used in training
])

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Preprocess the image
input_image = transform(image)
input_image = input_image.unsqueeze(0)  # Add batch dimension

# Load your trained model
net = Net()
net.load_state_dict(torch.load(PATH))
net.eval()  # Set to evaluation mode

# Run the image through the network and apply softmax
with torch.no_grad():
    output = net(input_image)
    probabilities = F.softmax(output, dim=1)[0]  # Apply softmax and get first (and only) item

# Display the scores for each class
for i, class_name in enumerate(classes):
    print(f"{class_name}: {probabilities[i].item():.6f}")

In [ ]:
#plot the image
imshow(input_image[0]) 